#### Importing Dependencies

In [1]:
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import sys
import pandas

Using Theano backend.
Using gpu device 0: GeForce GT 730M (CNMeM is disabled, cuDNN 5103)


#### Defining WordVec Parameters

In [2]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

#### Importing Word Vectors

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


#### Generating a list of sentences

In [4]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
sentences = [] # List of list of sentences.
sent = corpus.findall('.//sentence')
for s in sent:
    sentences.append(s.find('text').text)

print ('Generated list of sentences..')
# print (type(sentences[0].find('text').text))
# print (sent[0])

# texts = []  # list of text samples
# labels_index = {}  # dictionary mapping label name to numeric id
# labels = []  # list of label ids
# for name in sorted(os.listdir('20news-bydate/20news-bydate-train/')):
#     path = os.path.join('20news-bydate/20news-bydate-train/', name)
#     if os.path.isdir(path):
#         label_id = len(labels_index)
#         labels_index[name] = label_id
#         # mapping a document to a label
#         for fname in sorted(os.listdir(path)):
#             if fname.isdigit():
#                 fpath = os.path.join(path, fname)
#                 f = open(fpath)
#                 texts.append(f.read())
#                 f.close()
#                 labels.append(label_id)
#                 # mapping a label id to all the documents in a directory.

# print('Found %s texts.' % len(texts))
# # print (labels[:1000])
# print (texts[0])
# print (sentences[0])

Processing text dataset
Generated list of sentences..


#### vectorize the text samples into a 2D integer tensor and padding the sentences

In [5]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, lower=False)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5250 unique tokens.


In [6]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) # Training Inputs
print (type(sequences))
print (type(data))

<type 'list'>
<type 'numpy.ndarray'>


#### Preparing Embedding Matrix

In [7]:
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, 300))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [8]:
embedding_layer = Embedding(
                            nb_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False
                           )

#### Defining output data for Training

In [9]:
import nltk
raw_output = corpus.findall('.//sentence')
train_out=[]
for output in raw_output:
    s = nltk.word_tokenize(output.find('text').text)
    tags_for_sent = nltk.pos_tag(s)
    ind = [0] * len(s)
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    ind[s.index(aspect_term.attrib['term'])] = 1
                except:
                    continue
    train_out.append(ind)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


#### view training data and corresponding outputs

In [11]:
print (data[0])
print (train_out[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

# Defining and Training the model

In [12]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.preprocessing import sequence
from keras.optimizers import *
from keras.regularizers import l2

In [13]:
# model = Sequential()

# model.add(Convolution1D(100, 2, border_mode="same", input_shape=(80, 300)))
# model.add(Activation("tanh"))
# model.add(MaxPooling1D(pool_length=5))
# model.add(Convolution1D(50, 3, border_mode="same"))
# model.add(Activation("tanh"))
# model.add(MaxPooling1D(pool_length=2))
# model.add(Flatten())
# model.add(Dense(500))
# model.add(Activation("tanh"))
# # softmax classifier
# model.add(Dense(80, W_regularizer=l2(0.01)))
# model.add(Activation("softmax"))

# adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08)
# model.compile(loss='categorical_crossentropy',
#               optimizer='adadelta',
#               metrics=['accuracy'])
# print('Training model..')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(100, 2, activation='tanh', border_mode="same")(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(50, 3, activation='tanh',border_mode="same")(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(500, activation='tanh')(x)
preds = Dense(80, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print ('Model Trained..')

Model Trained..


In [14]:
print (model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1000, 300)     0           input_1[0][0]                    
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 1000, 100)     60100       embedding_1[0][0]                
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 200, 100)      0           convolution1d_1[0][0]            
___________________________________________________________________________________________

In [47]:
model.fit(data, train_out,
          validation_split=0.1,
          batch_size=10,
          nb_epoch=10
         )

Exception: Error when checking model target: expected dense_11 to have shape (None, 80) but got array with shape (3044, 1)